In [70]:
# basic libraries you need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The following is code for uploading a file to the colab.research.google 
# environment.

# library for uploading files
from google.colab import files 

def upload_files():
    # initiates the upload - follow the dialogues that appear
    uploaded = files.upload()

    # verify the upload
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

    # uploaded files need to be written to file to interact with them
    # as part of a file system
    for filename in uploaded.keys():
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])

# Data Preparation

This assignment is about preparing data for machine learning experiments. We have given you a dataset called "Melbourne_house_FULL.csv". You can find out about this dataset here:

https://www.kaggle.com/anthonypino/melbourne-housing-market

The dataset is for a housing price prediction task. The price column is the target value and the rest of the columns are features. We will request a series of cleaning tasks from you. The final result should be a dataset ready to do ML with.

Load the dataset using the code below. Notice that some columns are dropped. This is intentional as some of the columns in the original dataset are difficult to work with.


In [ ]:
upload_files()

In [71]:
house_df = pd.read_csv("Melbourne_housing_FULL.csv")
house_df = house_df.drop(["Suburb", "Address", "Date", "Postcode", "SellerG", "CouncilArea", "Lattitude", "Longtitude"], axis=1)

In [72]:
#Run EDA
house_df.head()

,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
0,2,h,NaN,SS,2.5,2.0,1.0,1.0,126.0,NaN,NaN,Northern Metropolitan,4019.0
1,2,h,1480000.0,S,2.5,2.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
2,2,h,1035000.0,S,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
3,3,u,NaN,VB,2.5,3.0,2.0,1.0,0.0,NaN,NaN,Northern Metropolitan,4019.0
4,3,h,1465000.0,SP,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,4019.0


In [73]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          34857 non-null  int64  
 1   Type           34857 non-null  object 
 2   Price          27247 non-null  float64
 3   Method         34857 non-null  object 
 4   Distance       34856 non-null  float64
 5   Bedroom2       26640 non-null  float64
 6   Bathroom       26631 non-null  float64
 7   Car            26129 non-null  float64
 8   Landsize       23047 non-null  float64
 9   BuildingArea   13742 non-null  float64
 10  YearBuilt      15551 non-null  float64
 11  Regionname     34854 non-null  object 
 12  Propertycount  34854 non-null  float64
dtypes: float64(9), int64(1), object(3)
memory usage: 3.5+ MB


### 1) Remove Null prices

Remove any row where the Price column is null. Remember, if you are scared of messing with the data while testing ideas make a copy of it by calling `.copy` on the original dataframe. For the rest of the exercise, assume that operations should be done on the dataframe with no null Prices.

In [74]:
#house_df.Feature.notnull()
#house_df.Feature.isnull()
#house_df.loc[]
# code goes here

house_df.Price.isna()
print(house_df.Price.isna().sum())

7610


In [75]:
house_df = house_df.dropna(axis = 0, subset = ['Price'])
print(house_df.Price.isna().any())

house_df

False


,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
1,2,h,1480000.0,S,2.5,2.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
2,2,h,1035000.0,S,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
4,3,h,1465000.0,SP,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,4019.0
5,3,h,850000.0,PI,2.5,3.0,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,4019.0
6,4,h,1600000.0,VB,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,4,h,1480000.0,PI,6.3,4.0,1.0,3.0,593.0,NaN,NaN,Western Metropolitan,6543.0
34853,2,h,888000.0,SP,6.3,2.0,2.0,1.0,98.0,104.0,2018.0,Western Metropolitan,6543.0
34854,2,t,705000.0,S,6.3,2.0,1.0,2.0,220.0,120.0,2000.0,Western Metropolitan,6543.0
34855,3,h,1140000.0,SP,6.3,NaN,NaN,NaN,NaN,NaN,NaN,Western Metropolitan,6543.0


### 2) Replace null "Regionname" values with the most frequent region name

Don't use Imputer objects for this, as sklearn's Imputer objects are not equipped to handle categorical data.

Hint: You can get the name of the most frequent value in a series by going `series_object.value_counts.index[0]`.

In [76]:
house_df.Regionname.describe()

count                     27244
unique                        8
top       Southern Metropolitan
freq                       8524
Name: Regionname, dtype: object

In [77]:
house_df.Regionname.unique()

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria', nan], dtype=object)

In [78]:
house_df.Regionname.value_counts().index[0]

'Southern Metropolitan'

In [79]:
house_df.Regionname = house_df.Regionname.replace(np.nan, house_df.Regionname.value_counts().index[0])

In [80]:
house_df.Regionname.describe()

count                     27247
unique                        8
top       Southern Metropolitan
freq                       8527
Name: Regionname, dtype: object

In [81]:
house_df.Regionname.unique()

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

### 3) Impute the numerical columns with the mean value of that column

Use the Imputer class with default arguments to do this. We started you off by listing the numerical columns.

In [86]:
#from sklearn.preprocessing import Imputer

from sklearn.impute import SimpleImputer

num_columns = ["Landsize",
               "Distance",
               "BuildingArea",
               "Propertycount"]


house_df[num_columns].isna().sum()

Landsize          9265
Distance             1
BuildingArea     16591
Propertycount        3
dtype: int64

In [98]:
imp_mean = SimpleImputer()

#imp_mean.fit()

imp_mean.fit(house_df[num_columns])
house_df[num_columns] = imp_mean.transform(house_df[num_columns])

house_df[num_columns].isna().sum()

Landsize         0
Distance         0
BuildingArea     0
Propertycount    0
dtype: int64

### 4) Impute the integer columns with the most frequent value of that column

Use the Imputer class with `strategy=most_frequent`. We started you off by listing the integer columns.

In [99]:
from sklearn.impute import SimpleImputer


int_columns = ["Bedroom2",
               "Bathroom",
               "Car",
               "YearBuilt"]


house_df[int_columns].isna().sum()


Bedroom2      6441
Bathroom      6447
Car           6824
YearBuilt    15163
dtype: int64

In [101]:
imp_most_freq = SimpleImputer( strategy='most_frequent')

imp_most_freq.fit(house_df[int_columns])

house_df[int_columns] = imp_most_freq.transform(house_df[int_columns])

house_df[int_columns].isna().sum()

Bedroom2     0
Bathroom     0
Car          0
YearBuilt    0
dtype: int64

### 5) Discretize the BuildingArea Column by making a new column named BuildingAreaDiscrete

More specifically, make a new column that has three new categories "small", "medium", "large". We listed the labels to use below.

Use the pd.qcut function to do this. Note: You may have to use the argument `duplicates="drop"` if you are getting an error.


In [102]:
building_labels = ["small", "medium", "large"]
new_column_name = "BuildingAreaDiscrete"
# code goes here

In [110]:
house_df[new_column_name] = pd.qcut(house_df.BuildingArea, 4, labels = building_labels, duplicates = "drop")

In [112]:
house_df.head(3)

,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount,BuildingAreaDiscrete
1,2,h,1480000.0,S,2.5,2.0,1.0,1.0,202.0,156.834586,1970.0,Northern Metropolitan,4019.0,medium
2,2,h,1035000.0,S,2.5,2.0,1.0,0.0,156.0,79.000000,1900.0,Northern Metropolitan,4019.0,small
4,3,h,1465000.0,SP,2.5,3.0,2.0,0.0,134.0,150.000000,1900.0,Northern Metropolitan,4019.0,small


### 6) Make dummy variables of the categorical columns

We use pd.get_dummies to do this, but if you are comfortable with a different technique go ahead and try that. We identified that categorical columns we want "dummified" for you.

In [113]:
cat_columns = ["Type", "Method", "Regionname", "BuildingAreaDiscrete"]

# code goes here

house_df[cat_columns]


,Type,Method,Regionname,BuildingAreaDiscrete
1,h,S,Northern Metropolitan,medium
2,h,S,Northern Metropolitan,small
4,h,SP,Northern Metropolitan,small
5,h,PI,Northern Metropolitan,medium
6,h,VB,Northern Metropolitan,small
...,...,...,...,...
34852,h,PI,Western Metropolitan,medium
34853,h,SP,Western Metropolitan,small
34854,t,S,Western Metropolitan,small
34855,h,SP,Western Metropolitan,medium


In [115]:
one_hot = pd.get_dummies(house_df[cat_columns])

In [150]:
house_df_fork_6 = house_df.copy()

#just for trialing one hot encoding later on...



In [151]:
house_df = house_df.drop(cat_columns, axis = 1)

In [152]:
house_df = house_df.join(one_hot)

In [ ]:
#later on... might not be functional, just messing around

from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder

house_df_sk = house_Df_fork_6.copy()

enc = OneHotEncoder()
one_hot = enc.fit_transform()


### 7)  Remove the top 1% of Prices

Remove rows that have a Price in the top 1% of prices. This corresponds prices above the 99th percentile. Use the `quantile` method to accomplish this.

In [ ]:
# code goes here

In [153]:
house_df.Price.describe()

count    2.724700e+04
mean     1.050173e+06
std      6.414671e+05
min      8.500000e+04
25%      6.350000e+05
50%      8.700000e+05
75%      1.295000e+06
max      1.120000e+07
Name: Price, dtype: float64

In [154]:
price_99q = house_df.Price.quantile(q=0.99)
price_99q

3400540.000000001

In [155]:
house_df = house_df[house_df.Price < price_99q]

In [156]:
house_df.Price.describe()

count    2.697400e+04
mean     1.016466e+06
std      5.406984e+05
min      8.500000e+04
25%      6.331250e+05
50%      8.650000e+05
75%      1.275000e+06
max      3.400000e+06
Name: Price, dtype: float64

### 8) Engineer a new column called "BathroomRatio"

Make a new column named "BathroomRatio". This column is the number of bathrooms divided by the number of rooms. This gives an idea of the number of bathrooms in proportion to the size of the house.

In [ ]:
# Code goes here

In [157]:
house_df.columns

Index(['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'Propertycount', 'Type_h', 'Type_t',
       'Type_u', 'Method_PI', 'Method_S', 'Method_SA', 'Method_SP',
       'Method_VB', 'Regionname_Eastern Metropolitan',
       'Regionname_Eastern Victoria', 'Regionname_Northern Metropolitan',
       'Regionname_Northern Victoria', 'Regionname_South-Eastern Metropolitan',
       'Regionname_Southern Metropolitan', 'Regionname_Western Metropolitan',
       'Regionname_Western Victoria', 'BuildingAreaDiscrete_small',
       'BuildingAreaDiscrete_medium', 'BuildingAreaDiscrete_large'],
      dtype='object')

In [158]:
house_df['BathroomRatio'] = house_df['Bathroom']/house_df['Rooms']

In [159]:
house_df.head()

,Rooms,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount,Type_h,Type_t,Type_u,Method_PI,Method_S,Method_SA,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria,BuildingAreaDiscrete_small,BuildingAreaDiscrete_medium,BuildingAreaDiscrete_large,BathroomRatio
1,2,1480000.0,2.5,2.0,1.0,1.0,202.0,156.834586,1970.0,4019.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.500000
2,2,1035000.0,2.5,2.0,1.0,0.0,156.0,79.000000,1900.0,4019.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.500000
4,3,1465000.0,2.5,3.0,2.0,0.0,134.0,150.000000,1900.0,4019.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0.666667
5,3,850000.0,2.5,3.0,2.0,1.0,94.0,156.834586,1970.0,4019.0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.666667
6,4,1600000.0,2.5,3.0,1.0,2.0,120.0,142.000000,2014.0,4019.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0.250000


### 9) Separate the Price column from the other features

Make two new variables:
1) A Series that contains just the Price column
2) A DataFrame that contains every other feature
Use reasonable names for these two new variables

In [161]:
# Code goes here

house_price = house_df['Price']

house_features = house_df.copy().drop('Price', axis = 1)


In [165]:
house_features.head(5)

,Rooms,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount,Type_h,Type_t,Type_u,Method_PI,Method_S,Method_SA,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria,BuildingAreaDiscrete_small,BuildingAreaDiscrete_medium,BuildingAreaDiscrete_large,BathroomRatio
1,2,2.5,2.0,1.0,1.0,202.0,156.834586,1970.0,4019.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.500000
2,2,2.5,2.0,1.0,0.0,156.0,79.000000,1900.0,4019.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.500000
4,3,2.5,3.0,2.0,0.0,134.0,150.000000,1900.0,4019.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0.666667
5,3,2.5,3.0,2.0,1.0,94.0,156.834586,1970.0,4019.0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.666667
6,4,2.5,3.0,1.0,2.0,120.0,142.000000,2014.0,4019.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0.250000


### 10) Divide the data into a train and test set

Use `train_test_split` from `sklearn.model_selection` to accomplish this. Use a test size of 10%. Use the variables you made in number **9)** to accomplish this.

In [ ]:
# Code goes here

In [166]:
from sklearn.model_selection import train_test_split

In [167]:
X_train, X_test, y_train, y_test = train_test_split(house_features, house_price, test_size=0.10)

In [169]:
X_train.size

704004

In [170]:
X_test.size

78242

### 11) Scale the data

Scale the features using standardization. You can use whatever technique for doing this.

Bonus: Scale just using the training features and then transform (scale) the test features using the mean and stddev learned from the training features. This can be done most efficiently with the `StandardScaler` class in `sklearn.preprocessing`. 

In [ ]:
# Code goes here

In [171]:
from sklearn.preprocessing import StandardScaler

In [186]:

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(X_train)
np_standardized_train = scaler.transform(X_train)
np_standardized_test = scaler.transform(X_test)

In [187]:
X_train_df_standardized = pd.DataFrame(np_standardized, columns = list(X_train.columns) )

In [188]:
X_test_df_standardized = pd.DataFrame(np_standardized, columns = list(X_test.columns))